# ▶️ CUDA setup

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!nvidia-smi

Mon Sep 12 12:22:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## NVCC Plugin for Jupyter notebook
 




In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-le98z6zk
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-le98z6zk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=13aa44e33023324ee505691530497147e4c8518030a7d8b0717349d941047ce9
  Stored in directory: /tmp/pip-ephem-wheel-cache-4xcz8go5/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Bash and data setup

In [5]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

Overwriting /root/.bashrc


In [6]:
!source /root/.bashrc

In [7]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

Cloning into 'GPUcomputing'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 219 (delta 79), reused 189 (delta 49), pack-reused 0
Receiving objects: 100% (219/219), 1.30 MiB | 16.21 MiB/s, done.
Resolving deltas: 100% (79/79), done.


# PCA


La tecnica PCA per l'estrazione di componenti pricipali da un dataset contenente una grande quantità di rumore avviene attraverso l'elaborazione di matrici per ottenere un cambiamento di base, ovvero un punto di vista, dei dati stessi. Una matrice che rappresenta un dataset da elaborare sarà costituita da righe corrispondenti al numero di tipi di misurazioni e da colonne corrispondenti al numero di misurazioni. Le strategie adottate sono due:

- PCA attraverso il calcolo di autovalori e autovettori della matrice di covarianza; gli autovettori saranno i componenti principali

- PCA attraverso il calcolo SVD da cui si ottiene una matrice ortonormale corrispondente agli autovettori della matrice di covarianza

L'elaborazione è effettuata attraverso librerie CuBlas e CuSolver per il calcolo di prodotti matriciali, autovettori e SVD. 
Viene inoltre calcolato il tempo di esecuzione dei singoli passi dell'elaborazione e le due strategie vengono confrontate.

In [8]:
%%writefile src/pca.cu

#include <stdio.h>
#include <stdlib.h>

#include "../GPUcomputing/utils/common.h"
#include <cuda_runtime.h>
#include <cusparse.h>
#include <cuda.h>
#include "cublas_v2.h"
#include <cusolverDn.h>

#define BLOCKS  24
#define R 2000   //righe corrispondenti al numero di tipi di misurazioni
#define C 300    //colonne corrispondenti al numero di misurazioni
#define P 5     //numero di Principal components a cui si è interessati

void generateMatrix(int, int, float**);
float* orderInversion(int, float*);
double var(int, int, float*);
__global__ void meanSubtraction(float*, float*, int, int);
double serialPCA(float*);
void transpose(float*, int, int, float*);
void multiply(float*, int, int, float*, int, int, float*);
float maxind(float*, int, int);
void update(int, float, float*, bool*, int*);
void rotate(int, int, int, int, float*, double, double);
void compute_V(float*, float*, float**, float**);
void initialize_identity(float*, int);
void mergesort(float*, int, int*, int, int);
void merge(float*, int*, int, int, int);

int main (int argc, char *argv[]) {
    
    int height = R;
    int width = C;
    int size = height * width;
    float alpha;
    float beta = 0.0f;
    float executionTime = 0.0f;

    float *matrix, *covariance, *principal_components;
    float *eigenvalues, *eigenvectors;
    float *d_matrix, *d_C;
    float *d_eigenval, *d_eigenvect, *d_work;
    float *d_rwork = nullptr;

    //Matrici ausiliarie per il calcolo di SVD
    float *U, *S, *VT;
    float *d_U, *d_S, *d_VT;

    cudaStream_t stream;

    cublasHandle_t handle;

    cusolverDnHandle_t cusolverH;
    int lwork = 0;
    int *d_info;
    int info = 0;

    cudaEvent_t start, stop;
	  cudaEventCreate(&start);
	  cudaEventCreate(&stop);


    //Generazione matrice di input, calcolo della media e sottrazione di essa da ciascun elemento dell'input

    generateMatrix(height, width, &matrix);

    CHECK(cudaMalloc((void **)&d_matrix, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_C, size * sizeof(float)));
    CHECK(cudaMemcpy(d_matrix, matrix, size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block(BLOCKS, BLOCKS);
	  dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);
    meanSubtraction<<<grid, block>>>(d_matrix, d_C, width, height);
    CHECK(cudaDeviceSynchronize());

	  CHECK(cudaMemcpy(matrix, d_C, size * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_C));
  
    //Calcolo del prodotto della matrice per la sua trasposta; Sgemm ovvero alpha * A * A_Trasposta + beta * C 
    //Che simula il calcolo di matrice di covarianza S = 1/n-1 * A * A_Trasposta

    alpha = 1 / (float)(width - 1);

    CHECK_CUBLAS(cublasCreate(&handle));
    CHECK(cudaMalloc((void **)&d_matrix, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_C, height * height * sizeof(float)));
    CHECK(cudaMemset(d_C, 0, height * height *sizeof(float)));
    CHECK_CUBLAS(cublasSetMatrix(height, width, sizeof(float), matrix, height, d_matrix, height));

    cudaEventRecord(start);
    CHECK_CUBLAS(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_T, height, width, width, &alpha, d_matrix, height, d_matrix, height, &beta, d_C, height));
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds;
	  cudaEventElapsedTime(&milliseconds, start, stop);
	  printf(" Matrix product for its transpose took: %.5f (sec)\n\n", milliseconds / 1000.0);
    executionTime += milliseconds;

    covariance = (float *)malloc (height * height * sizeof(float));
    CHECK_CUBLAS(cublasGetMatrix(height, height, sizeof(float), d_C, height, covariance, height));

    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_C));

    //Calcolo Autovalori 

    eigenvalues = (float *)malloc (height * sizeof(float));
    eigenvectors = (float *)malloc (height * height * sizeof(float));

    cusolverDnCreate(&cusolverH);
    cudaStreamCreate(&stream);
    cusolverDnSetStream(cusolverH, stream);

    CHECK(cudaMalloc((void **)&d_matrix, height * height * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_eigenval, height * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_info, sizeof(int)));

    cusolverDnSsyevd_bufferSize(cusolverH, CUSOLVER_EIG_MODE_VECTOR, CUBLAS_FILL_MODE_LOWER, height, d_matrix, height, d_eigenval, &lwork);
    CHECK(cudaMalloc((void **)&d_work, sizeof(float) * lwork));
    CHECK(cudaMemcpyAsync(d_matrix, covariance, sizeof(float) * height * height, cudaMemcpyHostToDevice, stream));

    cudaEventRecord(start);
    cusolverDnSsyevd(cusolverH, CUSOLVER_EIG_MODE_VECTOR, CUBLAS_FILL_MODE_LOWER, height, d_matrix, height, d_eigenval, d_work, lwork, d_info);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&milliseconds, start, stop);
	  printf(" Eigenvalues and Eigenvectors calculation took: %.5f (sec)\n\n", milliseconds / 1000.0);
    executionTime += milliseconds;

    CHECK(cudaMemcpyAsync(eigenvalues, d_eigenval, sizeof(float) * height, cudaMemcpyDeviceToHost, stream));
    CHECK(cudaMemcpyAsync(eigenvectors, d_matrix, sizeof(float) * height * height, cudaMemcpyDeviceToHost, stream));
    CHECK(cudaMemcpyAsync(&info, d_info, sizeof(int), cudaMemcpyDeviceToHost, stream));

    CHECK(cudaStreamSynchronize(stream));

    //Controllo la correttezza dell'operazione

    if (info < 0) {
            printf("%d-th parameter is wrong \n", info);
            return 0;
        }

    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_eigenval));
    CHECK(cudaFree(d_info));
    CHECK(cudaFree(d_work));

    cusolverDnDestroy(cusolverH);
    CHECK(cudaStreamDestroy(stream));

    //Ordino gli Autovalori in ordire decrescente

    eigenvalues = orderInversion(height, eigenvalues);

    //Calcolo i Principal Components: T =  eigenvectors * matrix

    principal_components = (float *)malloc (height * width * sizeof(float));

    CHECK_CUBLAS(cublasCreate(&handle));
    CHECK(cudaMalloc((void **)&d_matrix, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_eigenvect, height * height * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_C, width * height * sizeof(float)));
    CHECK(cudaMemset(d_C, 0, width * height *sizeof(float)));
    CHECK_CUBLAS(cublasSetMatrix(height, width, sizeof(float), matrix, height, d_matrix, height));
    CHECK_CUBLAS(cublasSetMatrix(height, height, sizeof(float), eigenvectors, height, d_eigenvect, height));

    alpha = 1.0f;
    cudaEventRecord(start);
    CHECK_CUBLAS(cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, height, width, height, &alpha, d_eigenvect, height, d_matrix, height, &beta, d_C, height));
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&milliseconds, start, stop);
	  printf(" Principal Components calculation took: %.5f (sec)\n\n", milliseconds / 1000.0);
    executionTime += milliseconds;

    CHECK_CUBLAS(cublasGetMatrix(height, width, sizeof(float), d_C, height, principal_components, height));

    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_C));
    CHECK(cudaFree(d_eigenvect));

    //Valuto quanti Principal components tenere per la riduzione, il parametro r descrive quanta varianza si è interessati a tenere

    int r = P;
   
    double variance = var(r, height, eigenvalues);

    printf(" With a parameter of %d elements we were able to keep %f percentage of principal components\n", r, variance);
    printf(" PCA technique through eigenvalues calculation took in total %f seconds\n\n", executionTime / 1000.0);
    float executionTimeSVD = 0.0f;

    free(covariance);
    free(eigenvalues);
    free(eigenvectors);
    free(principal_components);

    //Calcolo ora la SVD

    cusolverDnCreate(&cusolverH);
    cudaStreamCreate(&stream);
    cusolverDnSetStream(cusolverH, stream);

    U = (float *)malloc (height *height * sizeof(float));
    S = (float *)malloc (size * sizeof(float));
    VT = (float *)malloc (size * sizeof(float));

    CHECK(cudaMalloc((void **)&d_matrix, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_S,size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_U, height *  height * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_VT, height * width * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_info, sizeof(int)));

    cusolverDnSgesvd_bufferSize(cusolverH, height, width, &lwork);
    CHECK(cudaMalloc((void **)&d_work, sizeof(float) * lwork));
    CHECK(cudaMemcpyAsync(d_matrix, matrix, sizeof(float) * size, cudaMemcpyHostToDevice, stream));

    signed char jobu = 'A'; 
    signed char jobvt = 'A';

    cudaEventRecord(start);
    cusolverDnSgesvd(cusolverH, jobu, jobvt, height, width, d_matrix, height, d_S, d_U, height, d_VT, height, d_work, lwork, d_rwork, d_info);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&milliseconds, start, stop);
	  printf(" SVD calculation took: %.5f (sec)\n", milliseconds / 1000.0);
    executionTimeSVD += milliseconds;

    CHECK(cudaMemcpyAsync(U, d_U, sizeof(float) * height * height, cudaMemcpyDeviceToHost, stream));
    CHECK(cudaMemcpyAsync(VT, d_VT, sizeof(float) * size, cudaMemcpyDeviceToHost, stream));
    CHECK(cudaMemcpyAsync(S, d_S, sizeof(float) * size, cudaMemcpyDeviceToHost, stream));
    CHECK(cudaMemcpyAsync(&info, d_info, sizeof(int), cudaMemcpyDeviceToHost, stream));

    CHECK(cudaStreamSynchronize(stream));

    if (info < 0) {
        printf("%d-th parameter is wrong\n", info);
        return 0;
    }
    else if (info == 0)
        printf(" SVD calculation converges\n\n");
    else
        printf("With info = %d SVD does not converge\n", info);

 //I valori in S sono le singular value decomposition della matrice matrix ordinate in ordine decrescente
 //I valori in VT sono equivalenti agli autovettori del processo pca standard

    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_U));
    CHECK(cudaFree(d_S));
    CHECK(cudaFree(d_VT));
    CHECK(cudaFree(d_info));
    CHECK(cudaFree(d_work));
    CHECK(cudaFree(d_rwork));

    cusolverDnDestroy(cusolverH);
    CHECK(cudaStreamDestroy(stream));

    //Calcolo i Principal Components: T = matrix * VT

    principal_components = (float *)malloc (height * height * sizeof(float));

    CHECK_CUBLAS(cublasCreate(&handle));
    CHECK(cudaMalloc((void **)&d_matrix, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_VT, size * sizeof(float)));
    CHECK(cudaMalloc((void **)&d_C, height * height * sizeof(float)));
    CHECK(cudaMemset(d_C, 0, height * height *sizeof(float)));
    CHECK_CUBLAS(cublasSetMatrix(height, width, sizeof(float), matrix, height, d_matrix, height));
    CHECK_CUBLAS(cublasSetMatrix(height, width, sizeof(float), VT, height, d_VT, height));

    alpha = 1.0f;
    cudaEventRecord(start);
    CHECK_CUBLAS(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, height, width, width, &alpha, d_matrix, height, d_VT, height, &beta, d_C, height));
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&milliseconds, start, stop);
	  printf(" Principal Components calculation took: %.5f (sec)\n\n", milliseconds / 1000.0);
    executionTimeSVD += milliseconds;

    CHECK_CUBLAS(cublasGetMatrix(height, height, sizeof(float), d_C, height, principal_components, height));

    CHECK(cudaFree(d_matrix));
    CHECK(cudaFree(d_C));
    CHECK(cudaFree(d_VT));

    //Valuto quanti Principal components tenere per la riduzione, il parametro r descrive quanta varianza si è interessati a tenere

    variance = 0.0;
    variance = var(r, width, S);

    printf(" With a parameter of %d elements we were able to keep %f percentage of principal components\n", r, variance);
    printf(" PCA technique through SVD calculation took in total %f seconds\n\n", executionTimeSVD / 1000.0);

    printf(" Speedup between classic PCA and SVD is :  %.4f\n\n", executionTime/executionTimeSVD);

    printf(" Calculating PCA through serial algorithm...\n");
    double timeElapsed = serialPCA(matrix);
    printf(" Serial PCA took %f\n", timeElapsed);
    printf(" GPU execution of the PCA algorithm allows a speedup of %.4f when using classic PCA and of %.4f when using SVD\n", timeElapsed/executionTime * 1000, timeElapsed/executionTimeSVD * 1000);

    return 0;
}

void generateMatrix(int rows, int cols, float **matrix) {
	float *temp = (float *) malloc(sizeof(float) * rows * cols);

  for (int i = 0; i < cols; i++)
    for (int j = 0; j < rows; j++){
      temp[i * rows + j] = (float)rand() / (RAND_MAX) * 10;
    }
	*matrix = temp;
}

float* orderInversion(int dim, float *eigenvalues){
  float *temp = (float *)malloc (sizeof(float)*dim);

  for(int i = 0; i < dim; i++)
    temp[i] = eigenvalues[dim - 1 - i];

  return temp;
}

double var(int r, int size, float *v) {

  double sum1 = 0.0;
  double sum2 = 0.0;

  for (int i = 0; i < r; i++)
    sum1 += v[i];

  for (int i = 0; i < size; i++)
    sum2 += v[i];

  double variance = sum1/sum2 *100;      

  if (variance < 0)
    variance = variance + 100;
  else if (variance > 100)
    variance = 200 - variance;

  return variance;

}

__global__ void meanSubtraction(float* matrix, float *output, int w, int h) {
	
	uint row = blockIdx.y * blockDim.y + threadIdx.y;
	uint col = blockIdx.x * blockDim.x + threadIdx.x;

	float sum = 0.0;
  float mean = 0.0;
  float val = 0.0;

	__shared__ float Ms[BLOCKS][BLOCKS];

	uint numBlocks = (w + BLOCKS - 1) / BLOCKS;
	for (int m = 0; m < numBlocks; m++) {
		
		uint c = m * BLOCKS + threadIdx.x;
		Ms[threadIdx.y][threadIdx.x] = matrix[row * w + c];

		__syncthreads();

		uint K = BLOCKS;
		if (m == numBlocks - 1) K = w - m * BLOCKS; 

    for (int k = 0; k < K; k++)
      for (int x = 0; x < K; x++)
        sum += Ms[k][x];

      mean = sum / (float)(w*h);
			val = Ms[threadIdx.y][threadIdx.x] - mean;

      __syncthreads();
	}

	if (row < w && col < h)
		output[row * h + col] = val;
}


double serialPCA (float *matrix){
  
  double start = seconds();
    
     int state = C, num_iter = 0, m, k, l; 
     float p, y, d, r, t;           
     double c, s;
     float *eigenvectors = (float *)malloc(sizeof(float)*C*C);
     initialize_identity(eigenvectors, C); 
     float *eigenvalues = (float *)malloc(sizeof(float) * C); 
     int *index = (int *)malloc(sizeof(int) * C);        
     bool *changed = (bool *)malloc(sizeof(bool) * C); 
     float *A_s = (float *)calloc(C * C, sizeof(float));
     float *matrix_T = (float *)malloc(sizeof(float) * C * R);
     transpose(matrix, R, C, matrix_T);
     multiply(matrix_T, C, R, matrix, R, C, A_s);

     for (int i = 0; i < C; i++){
         index[i] = maxind(A_s, C, i); 
         eigenvalues[i] = A_s[i * C + i];
         changed[i] = true;
       }
     while (state && num_iter < 10000000){         
         m = 0;
         for (int i = 1; i < C - 1; i++){          
             if (fabs(A_s[i * C + index[i]]) > fabs(A_s[m * C + index[m]]))
                 m = i;
         }
         k = m;
         l = index[k];
         p = A_s[k * C + l];
         y = 0.5 * (eigenvalues[l] - eigenvalues[k]);
         d = fabs(y) + sqrt(p * p + y * y);
         r = sqrt(p * p + d * d);
         c = d / r;
         s = p / r;
         t = p * p / d;
         if (y < 0){
             s = -s;
             t = -t;
         }
         A_s[k * C + l] = 0.0;
         update(k, -t, eigenvalues, changed, &state);
         update(l, t, eigenvalues, changed, &state);

         //rotate rows and cols k and l:
         for (int i = 0; i < k; i++){
             rotate(i, k, i, l, A_s, c, s);
         }
         for (int i = k + 1; i < l; i++){
             rotate(k, i, i, l, A_s, c, s);
         }
         for (int i = l + 1; i < C; i++){
             rotate(k, i, l, i, A_s, c, s);
         }
         //rotate eigenvectors:
         for (int i = 0; i < C; i++){
             float e_ik = c * eigenvectors[i * C + k] - s * eigenvectors[i * C + l];
             float e_il = s * eigenvectors[i * C + k] + c * eigenvectors[i * C + l];
             eigenvectors[i * C + k] = e_ik;
             eigenvectors[i * C + l] = e_il;
         }
         index[k] = maxind(A_s, C, k);
         index[l] = maxind(A_s, C, l);
         num_iter++;
     }
     //sort eigenvalues in desc:

     int *indices = (int *)malloc(sizeof(int) * C);
     for (int i = 0; i < C; i++)
         indices[i] = i;
     
     mergesort(eigenvalues, C, indices, 0, C - 1);

     //computing SIGMA:
     
     float *SIGMA = (float *)calloc(C * R, sizeof(float));
     float sum=0.0;
     for (int i = 0; i < C; i++){
         SIGMA[i] = sqrt(eigenvalues[i]);
         sum+=eigenvalues[i];     
     }
       
    //eigenvectors matrix (U for matrix_T*matrix):
   
     float *u_s = (float *) malloc(sizeof(float)*C*C);
     for (int row = 0; row < C; row++){
         for (int col = 0; col < C; col++)
             u_s[row * C + col] = eigenvectors[row * C + indices[col]];  
     }
     //compute V_T:
     float *V_T_s = (float *)calloc(R*R, sizeof(float));
     compute_V(SIGMA, matrix_T, &u_s, &V_T_s);     

     //compute serial PCA:
      int K_s=0;
      float retention_s = 0.0;
      int count_s = 0;
      while((retention_s < P) && (count_s < C)){
          retention_s+=(SIGMA[count_s]*SIGMA[count_s]/sum)*100; 
          K_s++;
          count_s++;
      }
      
      float *W_s = (float *)malloc(sizeof(float)*C*K_s);
      float *principal_components = (float *)malloc(sizeof(float)*R*K_s);
      for (int r=0; r<C; r++)
      {
          for (int c=0; c<K_s; c++)
          {
              W_s[r*K_s+c] = u_s[r*C+c];
          }
      }
     multiply(matrix, R, C, W_s, C, K_s, principal_components);
    
     return(seconds() - start);  
}

void initialize_identity(float *I, int size){
    memset(I, 0, sizeof(float)*size*size);
    for (int i = 0; i < size; i++)
        I[i * size + i] = 1.0;
}

void transpose(float *M, int m, int n, float *M_T){
    int i, j, index_;
    for (j=0; j<n; j++){
        index_ = j*m;
        for (i=0; i<m; i++)
            M_T[index_+i] = M[i*n+j];
    }
}

void multiply(float *M_1, int m1, int n1, float *M_2, int m2, int n2, float *result){
    
    float sum = 0.0;
   
    float *M_2_T = (float *)malloc(sizeof(float) * n2 * m2);
    transpose(M_2, m2, n2, M_2_T);
    int i, j, k, temp1, temp2;
    for (i = 0; i < m1; i++){
        temp1 = i * n1;
        for (j = 0; j < n2; j++){
            sum = 0.0;
            temp2 = j * m2;
            for (k = 0; k < n1; k++)
                sum += M_1[temp1 + k] * M_2_T[temp2 + k];
            result[i * n2 + j] = sum;
        }
    }
    free(M_2_T);
}

float maxind(float *A, int size, int k){
    int m = k + 1;
    for (int i = k + 2; i < size; i++){
        if (fabs(A[k * size + i]) > fabs(A[k * size + m]))
            m = i;
    }
    return m;
}

void update(int k, float t, float *e, bool *changed, int *state){
    float y = e[k];
    e[k] = y + t;
    if (changed[k] && (y == e[k])){
        changed[k] = false;
        (*state)--;
    }
    else if (!changed[k] && (y != e[k])){
        changed[k] = true;
        (*state)++;
    }
}

void rotate(int k, int l, int i, int j, float *A, double c, double s){
    float k_l = c * A[k * C + l] - s * A[i * C + j];
    float i_j = s * A[k * C + l] + c * A[i * C + j];
    A[k * C + l] = k_l;
    A[i * C + j] = i_j;
}

void compute_V(float *SIGMA, float *matrix_T, float **U, float **V_T){
    
    float *INV_SIGMA = (float *)calloc(R * C, sizeof(float)); 
    for (int i = 0; i < C; i++)
        INV_SIGMA[i * C + i] = 1.0 / (SIGMA[i]);
  
    float *U_T = (float *)malloc(sizeof(float) * C * C);
    transpose(*U, C, C, U_T);
    
    float *product = (float *)malloc(sizeof(float) * R * C);
    multiply(INV_SIGMA, R, C, U_T, C, C, product);
   
    multiply(product, R, C, matrix_T, C, R, *V_T);
    free(INV_SIGMA);
    free(U_T);
    free(product);
}

void merge(float *e, int *indices_e, int left_index, int mid, int right_index){
    int i = left_index, j = mid + 1, k = 0;
    float *sorted = (float *)malloc(sizeof(float) * (right_index - left_index + 1));
    int *sorted_indices = (int *)malloc(sizeof(int) * (right_index - left_index + 1));
    
    while (i <= mid && j <= right_index){
        if (fabs(e[i]) >= fabs(e[j])){
            sorted_indices[k] = indices_e[i];
            sorted[k++] = e[i++];
        }
        else{
            sorted_indices[k] = indices_e[j];
            sorted[k++] = e[j++];
        }
    }
    while (i <= mid){
        sorted_indices[k] = indices_e[i];
        sorted[k++] = e[i++];
    }
    while (j <= right_index){
        sorted_indices[k] = indices_e[j];
        sorted[k++] = e[j++];
    }
    
    memcpy(e + left_index, sorted, sizeof(float)*(right_index-left_index+1));
    memcpy(indices_e + left_index, sorted_indices, sizeof(int)*(right_index-left_index+1));
    free(sorted);
    free(sorted_indices);
}

void mergesort(float *e, int e_len, int *indices_e, int left_index, int right_index){
    if (left_index < right_index){
        int mid = (left_index + right_index) / 2;
        mergesort(e, e_len, indices_e, left_index, mid);
        mergesort(e, e_len, indices_e, mid + 1, right_index);
        merge(e, indices_e, left_index, mid, right_index);
    }
}


Writing src/pca.cu


In [ ]:
!nvcc -arch=sm_75 src/pca.cu -o pca -lcublas -lcusolver
!./pca

Il codice è stato testato con diversi valori per righe e colonne di matrice.

Con 20 righe e 10 colonne il codice sequenziale è il più performante di circa 5 volte per PCA con autovalori e 10 volte per SVD.

All'aumentare dei valori di righe e colonne il codice sequenziale diventa sempre più lento rispetto a quello parallelo, è stato testato con 2000 righe e 300 colonne un tempo di esecuzione di circa 2 minuti mentre PCA attraverso calcolo di autovalori o SVD si attestano intorno ai 0.2 e 0.03 secondi con un visibile vantaggio della tecnica SVD.

Per valori intermedi (è stato testato con 200 righe e 100 colonne) sarà invece la tecnica PCA con autovettori ad essere la più veloce.

Per concludere all'aumentare dei valori di righe e colonne con conseguente aumento di rilevazioni (di numeri) del sistema la tecnica parallela con SVD risulta la più performante.

In [ ]:
!nsys profile ./pca
!nsys stats ../content/report1.qdrep